In [1]:
import os
os.chdir("../")

In [2]:
from src.hf_dataset.dataset import get_banking_77

In [4]:
ds = get_banking_77()

In [6]:
from src.hf_models.get_model import get_distilbert
model, tokenizer, config = get_distilbert(task="SequenceClassification")

/home/tess/work/deep_learning/transformers/models
Model loaded successfully.


In [8]:

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
ds = ds.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
ds = ds.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [9]:
from src.config.configuration import ConfigurationManager
config_manager = ConfigurationManager()
params = config_manager.get_distilbert_config()

params

/home/tess/work/deep_learning/transformers/models


DistilBert(model_path='/home/tess/work/deep_learning/transformers/models/distilbert/model', tokenizer_path='/home/tess/work/deep_learning/transformers/models/distilbert/tokenizer', config_path='/home/tess/work/deep_learning/transformers/models/distilbert/config', model_name='distilbert')

In [10]:

from adapters import AutoAdapterModel

config.num_labels=77

model = AutoAdapterModel.from_pretrained(
    params.model_path,
    config=config,
)

Some weights of DistilBertAdapterModel were not initialized from the model checkpoint at /home/tess/work/deep_learning/transformers/models/distilbert/model and are newly initialized: ['heads.default.0.bias', 'heads.default.0.weight', 'heads.default.2.bias', 'heads.default.2.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.add_adapter("banking77", config="seq_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "banking77",
    num_labels=77)

# Activate the adapter
model.train_adapter("banking77")

In [12]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    compute_metrics=compute_accuracy,
)

In [13]:
trainer.train()

Step,Training Loss
200,3.966600


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=313, training_loss=3.6531968771840058, metrics={'train_runtime': 133.7001, 'train_samples_per_second': 74.817, 'train_steps_per_second': 2.341, 'total_flos': 212456455421280.0, 'train_loss': 3.6531968771840058, 'epoch': 1.0})